In [21]:
import google.generativeai as genai
import re
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from youtube_transcript_api import YouTubeTranscriptApi


GENAI_API_KEY = "a"
genai.configure(api_key=GENAI_API_KEY)


def extract_video_id(url):
    match = re.search(r"(?:v=|youtu\.be/|embed/|shorts/|v/)([a-zA-Z0-9_-]{11})", url)
    return match.group(1) if match else None


def get_video_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return " ".join([t["text"] for t in transcript])
    except Exception as e:
        return f"Error fetching transcript: {str(e)}"


def ask_question(transcript, question):
    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(f"Based on this transcript, answer the following question:\n\n{transcript}\n\nQuestion: {question}")

    if not response.text or "I'm sorry" in response.text or len(response.text.split()) < 5:
        return "❌ This video does not seem to contain that information."

    return response.text


class YouTubeChatbot:
    def __init__(self):
        self.transcript = None 
        self.video_url = ""

        # UI Elements
        self.video_url_widget = widgets.Text(
            value="",
            placeholder="Enter YouTube video URL",
            description="🔗 URL:",
            layout=widgets.Layout(width="80%"),
        )

        self.load_button = widgets.Button(
            description="Load Video",
            button_style="primary",
            tooltip="Click to fetch transcript",
            icon="download",
        )

        self.question_widget = widgets.Text(
            value="",
            placeholder="Ask a question...",
            description="🧐 Q:",
            layout=widgets.Layout(width="80%"),
        )

        self.ask_button = widgets.Button(
            description="Ask 🤖",
            button_style="success",
            tooltip="Click to ask a question",
            icon="search",
        )

        self.output = widgets.Output()


        self.load_button.on_click(self.load_video)
        self.ask_button.on_click(self.ask_question)


        display(self.video_url_widget, self.load_button, self.output, self.question_widget, self.ask_button)

    def load_video(self, b):
        with self.output:
            clear_output()
            self.video_url = self.video_url_widget.value.strip()
            video_id = extract_video_id(self.video_url)

            if not video_id:
                display(Markdown("❌ **Invalid YouTube URL.** Please enter a valid link."))
                return

            display(Markdown("📥 **Fetching transcript...**"))
            self.transcript = get_video_transcript(video_id)

            if "Error" in self.transcript:
                display(Markdown(f"❌ {self.transcript}"))
                return

            display(Markdown("✅ **Transcript loaded! You can now ask questions.**"))

    def ask_question(self, b):
        if not self.transcript:
            with self.output:
                display(Markdown("❌ **Please load a video first!**"))
            return

        question = self.question_widget.value.strip()
        if not question:
            with self.output:
                display(Markdown("❌ **Please enter a question!**"))
            return

        with self.output:
            display(Markdown(f"**🧐 You asked:** {question}"))
            display(Markdown("🤖 **Thinking...**"))

            answer = ask_question(self.transcript, question)
            display(Markdown(f"💡 **Answer:**\n\n{answer}"))


        self.question_widget.value = ""


chatbot = YouTubeChatbot()


Text(value='', description='🔗 URL:', layout=Layout(width='80%'), placeholder='Enter YouTube video URL')

Button(button_style='primary', description='Load Video', icon='download', style=ButtonStyle(), tooltip='Click …

Output()

Text(value='', description='🧐 Q:', layout=Layout(width='80%'), placeholder='Ask a question...')

Button(button_style='success', description='Ask 🤖', icon='search', style=ButtonStyle(), tooltip='Click to ask …